In [ ]:
!pip install flask pyngrok transformers torch torchvision

from flask import Flask, request, jsonify
from diffusers import StableDiffusionPipeline
from transformers import pipeline
import torch
import base64
import numpy as np
from io import BytesIO
from pyngrok import ngrok

# ngrok 인증키 설정
ngrok.set_auth_token("your_key_here")  # 여기에 ngrok 인증키를 입력하세요

app = Flask(__name__)

print("🚀 Emoseum Colab Server - Stable Diffusion & GoEmotions")
print("=" * 60)

In [ ]:
# Stable Diffusion 모델 로드
print("📥 Stable Diffusion 모델 로드 중...")
sd_pipeline = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16
)
sd_pipeline = sd_pipeline.to("cuda")
print("✅ Stable Diffusion 모델 로드 완료!")

# GoEmotions 모델 로드  
print("📥 GoEmotions 모델 로드 중...")
emotion_classifier = pipeline(
    "text-classification",
    model="joeddav/distilbert-base-uncased-go-emotions-student",
    device=0,  # GPU 사용
    top_k=None  # 모든 레이블의 점수를 반환
)
print("✅ GoEmotions 모델 로드 완료!")

print(f"🎯 Device set to use {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print("=" * 60)

# GoEmotions 감정 레이블과 VAD 매핑
EMOTION_LABELS = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval',
    'caring', 'confusion', 'curiosity', 'desire', 'disappointment',
    'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear',
    'gratitude', 'grief', 'joy', 'love', 'nervousness',
    'optimism', 'pride', 'realization', 'relief', 'remorse',
    'sadness', 'surprise', 'neutral'
]

# 최신 연구 기반 VAD 매핑 (Russell & Mehrabian 1977, Bradley & Lang 1999, Mohammad & Turney 2013)
# GoEmotions 28개 감정의 심리학적으로 검증된 VAD (Valence-Arousal-Dominance) 값
EMOTION_VAD_MAPPING = {
    # === 긍정 감정 12개 === #
    'admiration': (0.75, 0.45, 0.65),  # 존경 - 따뜻하고 차분하며 통제감 있음
    'amusement': (0.80, 0.65, 0.70),   # 재미 - 즐겁고 활기차며 통제된 즐거움
    'approval': (0.70, 0.35, 0.60),    # 승인 - 긍정적이고 차분하며 적당한 통제감
    'caring': (0.80, 0.40, 0.55),      # 돌봄 - 매우 긍정적이고 부드러우며 양육적 통제
    'excitement': (0.85, 0.85, 0.75),  # 흥분 - 높은 쾌락, 높은 에너지, 좋은 통제감
    'gratitude': (0.85, 0.45, 0.50),   # 감사 - 매우 긍정적이고 평화로우며 겸손함
    'joy': (0.90, 0.70, 0.75),         # 기쁨 - 최고 쾌락, 에너제틱, 자신감 있음
    'love': (0.90, 0.60, 0.60),        # 사랑 - 최고 쾌락, 따뜻한 에너지, 균형잡힌 통제
    'optimism': (0.75, 0.50, 0.70),    # 낙관 - 긍정적 전망, 적당한 에너지, 희망적 통제
    'pride': (0.80, 0.55, 0.80),       # 자부심 - 긍정적, 활기찬, 높은 통제/자신감
    'relief': (0.75, 0.25, 0.55),      # 안도 - 긍정적 해결, 진정시키는, 적당한 통제
    
    # === 부정 감정 11개 === #
    'anger': (0.10, 0.80, 0.60),       # 분노 - 매우 불쾌, 높은 각성, 가변적 통제
    'annoyance': (0.25, 0.60, 0.45),   # 짜증 - 불쾌, 적당히 각성, 제한된 통제
    'disappointment': (0.20, 0.45, 0.35), # 실망 - 불쾌, 의기소침, 낮은 통제
    'disapproval': (0.25, 0.50, 0.40), # 반대 - 불쾌, 활성화, 제한된 통제
    'disgust': (0.15, 0.55, 0.50),     # 혐오 - 매우 불쾌, 회피적 각성, 보호적 통제
    'embarrassment': (0.20, 0.65, 0.25), # 당황 - 불쾌, 활성화, 매우 낮은 통제
    'fear': (0.10, 0.75, 0.30),        # 두려움 - 매우 불쾌, 높은 각성, 낮은 통제
    'grief': (0.10, 0.50, 0.20),       # 슬픔(상실) - 매우 불쾌, 소진되지만 강렬, 매우 낮은 통제
    'nervousness': (0.30, 0.70, 0.25), # 불안 - 불쾌, 높은 각성, 낮은 통제
    'remorse': (0.15, 0.45, 0.25),     # 후회 - 매우 불쾌, 성찰적 각성, 낮은 통제
    'sadness': (0.15, 0.35, 0.25),     # 슬픔 - 매우 불쾌, 낮은 에너지, 낮은 통제
    
    # === 모호한 감정 4개 + surprise === #
    'confusion': (0.40, 0.55, 0.30),   # 혼란 - 중립-부정, 의문적 각성, 낮은 통제
    'curiosity': (0.60, 0.60, 0.55),   # 호기심 - 약간 긍정, 관여하는, 적당한 통제
    'desire': (0.65, 0.70, 0.50),      # 욕망 - 긍정적, 동기부여하는, 가변적 통제
    'realization': (0.55, 0.50, 0.60), # 깨달음 - 약간 긍정, 명료의 순간, 좋은 통제
    'surprise': (0.50, 0.75, 0.45),    # 놀람 - 중립, 높은 각성, 낮은 즉각적 통제
    
    # === 중립 1개 === #
    'neutral': (0.50, 0.50, 0.50)      # 중립 - 기준 중립 상태
}


def calculate_adaptive_threshold(emotions):
    """동적 threshold 계산 - 감정 점수 분포에 따라 조정"""
    if not emotions:
        return 0.1
    
    scores = [em['score'] for em in emotions]
    mean_score = np.mean(scores)
    max_score = max(scores)
    
    # 최고 점수가 낮으면 threshold도 낮게, 높으면 상대적으로 높게
    adaptive_threshold = max(0.1, min(0.4, mean_score * 0.6))
    
    return adaptive_threshold

def calculate_vad_scores(emotions, use_adaptive_threshold=True):
    """개선된 VAD 점수 계산 - 동적 threshold와 감정 강도 고려"""
    if not emotions:
        return [0.5, 0.5, 0.5]
    
    # 동적 threshold 사용
    if use_adaptive_threshold:
        threshold = calculate_adaptive_threshold(emotions)
    else:
        threshold = 0.3
    
    # 임계값 이상의 감정들만 선택
    filtered_emotions = [(em['label'], em['score']) for em in emotions if em['score'] >= threshold]
    
    if not filtered_emotions:
        # 임계값을 넘는 감정이 없으면 상위 3개 감정 사용
        sorted_emotions = sorted(emotions, key=lambda x: x['score'], reverse=True)
        filtered_emotions = [(em['label'], em['score']) for em in sorted_emotions[:3]]
    
    total_weight = sum(score for _, score in filtered_emotions)
    if total_weight == 0:
        return [0.5, 0.5, 0.5]
    
    weighted_vad = [0.0, 0.0, 0.0]
    
    # 감정 강도를 고려한 가중치 적용
    for emotion, score in filtered_emotions:
        vad = EMOTION_VAD_MAPPING.get(emotion, (0.5, 0.5, 0.5))
        # 점수에 제곱근을 적용해 극단값 완화
        adjusted_weight = np.sqrt(score)
        for i in range(3):
            weighted_vad[i] += vad[i] * adjusted_weight
    
    # 조정된 가중치의 총합으로 정규화
    adjusted_total_weight = sum(np.sqrt(score) for _, score in filtered_emotions)
    vad_scores = [v / adjusted_total_weight for v in weighted_vad]
    
    # 0-1 범위로 클리핑
    vad_scores = [max(0.0, min(1.0, v)) for v in vad_scores]
    
    return vad_scores

def calculate_intensity(emotions, threshold=0.3):
    """감정 강도 계산"""
    if not emotions:
        return "low"
    
    # 상위 3개 감정의 평균 점수
    top_emotions = emotions[:3]
    avg_score = np.mean([em['score'] for em in top_emotions])
    
    # Arousal 기반 강도 조정
    arousal_boost = 0.0
    for emotion in top_emotions:
        vad = EMOTION_VAD_MAPPING.get(emotion['label'], (0.5, 0.5, 0.5))
        arousal_boost += vad[1] * emotion['score']
    
    arousal_boost /= len(top_emotions)
    
    # 최종 강도 계산
    intensity_score = (avg_score + arousal_boost) / 2
    
    if intensity_score >= 0.7:
        return "high"
    elif intensity_score >= 0.4:
        return "medium"
    else:
        return "low"

@app.route('/generate', methods=['POST'])
def generate_image():
    """이미지 생성 엔드포인트"""
    global sd_pipeline
    
    if sd_pipeline is None:
        return jsonify({"success": False, "error": "Stable Diffusion model not loaded"})
    
    try:
        data = request.json
        prompt = data.get('prompt', '')
        
        if not prompt:
            return jsonify({"success": False, "error": "Prompt is required"})
        
        print(f"📝 이미지 생성 요청: {prompt[:50]}...")
        
        # 이미지 생성
        image = sd_pipeline(prompt).images[0]
        
        # Base64로 인코딩
        buffered = BytesIO()
        image.save(buffered, format="PNG")
        img_str = base64.b64encode(buffered.getvalue()).decode()
        
        print("✅ 이미지 생성 성공")
        
        return jsonify({
            "image": img_str, 
            "success": True,
            "service": "stable_diffusion",
            "generation_time": 30.0
        })
        
    except Exception as e:
        print(f"❌ 이미지 생성 실패: {e}")
        return jsonify({"success": False, "error": str(e)})

@app.route('/analyze_emotion', methods=['POST'])
def analyze_emotion():
    """
    GoEmotions 기반 감정 분석 엔드포인트
    
    작동 과정:
    1. 입력 텍스트를 GoEmotions 모델 (DistilBERT)에 전달
    2. 28개 감정 카테고리에 대한 확률 점수 계산
    3. 동적 threshold를 사용해 의미있는 감정들만 선별
    4. 심리학적으로 검증된 VAD 매핑을 통해 차원적 감정 값 계산
    5. 결과를 감정 키워드, VAD 점수, 신뢰도 등으로 구성하여 반환
    """
    global emotion_classifier
    
    if emotion_classifier is None:
        return jsonify({"success": False, "error": "GoEmotions model not loaded"})
    
    try:
        data = request.json
        text = data.get('text', '')
        threshold = data.get('threshold', 0.3)  # 기본값 (동적 threshold 사용 시 무시됨)
        
        if not text:
            return jsonify({"success": False, "error": "Text is required"})
        
        print(f"📝 감정 분석 요청: {text[:50]}...")
        
        # === 1단계: GoEmotions 모델을 통한 28개 감정 분류 === #
        # joeddav/distilbert-base-uncased-go-emotions-student 모델이
        # 입력 텍스트에 대해 28개 각 감정의 확률 점수를 계산
        results = emotion_classifier(text)
        all_emotions = results[0]  # pipeline은 리스트 안에 결과를 반환
        
        print(f"🔍 GoEmotions 원본 결과 (상위 5개): {[(em['label'], round(em['score'], 3)) for em in sorted(all_emotions, key=lambda x: x['score'], reverse=True)[:5]]}")
        
        # === 2단계: 감정 점수 수집 및 정렬 === #
        scores = {}
        emotions_list = []
        
        for result in all_emotions:
            label = result['label']
            score = result['score']
            scores[label] = score
            emotions_list.append({'label': label, 'score': score})
        
        # 점수 기준으로 정렬 (높은 점수부터)
        emotions_list.sort(key=lambda x: x['score'], reverse=True)
        
        # === 3단계: 동적 threshold 기반 의미있는 감정 선별 === #
        # 고정 threshold 대신 텍스트 감정 강도에 따라 적응적 조정
        adaptive_threshold = calculate_adaptive_threshold(all_emotions)
        print(f"🎯 동적 threshold: {adaptive_threshold:.3f} (기본값: {threshold})")
        
        # threshold 기반 필터링 (한 번만 적용)
        filtered_emotions = [em for em in emotions_list if em['score'] >= adaptive_threshold]
        
        # 필터링된 감정이 없으면 상위 5개 감정을 강제 선택
        if not filtered_emotions:
            filtered_emotions = emotions_list[:5]
            print("⚠️ threshold 기준 감정 없음 - 상위 5개 사용")
        else:
            # 상위 5개로 제한 (너무 많은 감정 방지)
            filtered_emotions = filtered_emotions[:5]
        
        emotion_keywords = [em['label'] for em in filtered_emotions]
        
        # 안전장치: 감정이 여전히 없으면 neutral 추가
        if not emotion_keywords:
            emotion_keywords = ['neutral']
            filtered_emotions = [{'label': 'neutral', 'score': 1.0}]
        
        print(f"✅ 선별된 감정들: {[(em['label'], round(em['score'], 3)) for em in filtered_emotions]}")
        
        # === 4단계: 심리학적 VAD 점수 계산 === #
        # Russell & Mehrabian 연구 기반 Valence-Arousal-Dominance 차원 계산
        vad_scores = calculate_vad_scores(all_emotions, use_adaptive_threshold=True)
        print(f"📊 VAD 점수: V={vad_scores[0]:.3f}, A={vad_scores[1]:.3f}, D={vad_scores[2]:.3f}")
        
        # === 5단계: 결과 메타데이터 계산 === #
        # 주요 감정 (가장 높은 점수)
        primary_emotion = emotion_keywords[0] if emotion_keywords else 'neutral'
        
        # 감정 강도 계산 (arousal 차원과 점수 강도 결합)
        emotional_intensity = calculate_intensity(all_emotions, adaptive_threshold)
        
        # 신뢰도 계산 (선별된 감정들의 평균 점수)
        confidence = float(np.mean([em['score'] for em in filtered_emotions]))
        
        print(f"🎯 주요 감정: {primary_emotion} (강도: {emotional_intensity}, 신뢰도: {confidence:.3f})")
        
        result_data = {
            "keywords": emotion_keywords,
            "vad_scores": vad_scores,
            "confidence": confidence,
            "primary_emotion": primary_emotion,
            "emotional_intensity": emotional_intensity,
            "all_scores": scores,
            "top_emotions": {em['label']: em['score'] for em in filtered_emotions}
        }
        
        print(f"✅ 감정 분석 성공: {emotion_keywords}")
        
        return jsonify({
            "success": True,
            "service": "goEmotions",
            **result_data
        })
        
    except Exception as e:
        print(f"❌ 감정 분석 실패: {e}")
        return jsonify({"success": False, "error": str(e)})

@app.route('/health', methods=['GET'])
def health_check():
    """헬스 체크 엔드포인트"""
    global sd_pipeline, emotion_classifier
    
    return jsonify({
        "status": "healthy",
        "stable_diffusion_loaded": sd_pipeline is not None,
        "goEmotions_loaded": emotion_classifier is not None,
        "gpu_available": torch.cuda.is_available(),
        "gpu_count": torch.cuda.device_count() if torch.cuda.is_available() else 0
    })

@app.route('/', methods=['GET'])
def home():
    """홈페이지"""
    return jsonify({
        "message": "Emoseum Colab Server - Stable Diffusion & GoEmotions",
        "endpoints": {
            "/generate": "POST - 이미지 생성 (prompt 필요)",
            "/analyze_emotion": "POST - 감정 분석 (text 필요)", 
            "/health": "GET - 헬스 체크"
        }
    })

In [ ]:
# ngrok 터널 생성
print("\n🌐 ngrok 터널 생성 중...")
public_url = ngrok.connect(5000)
print(f"\n{'='*60}")
print(f"🎯 Colab Server가 실행됩니다!")
print(f"🌐 Public URL: {public_url}")
print(f"{'='*60}\n")

# Flask 앱 실행
print("🚀 Flask 서버 시작...")
app.run(host='0.0.0.0', port=5000, debug=False)